In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

********* Required Libraries Imported *********


In [2]:
#load the saved model
loaded_model = xgb.Booster()
loaded_model.load_model('LeoPayu_BBILAP_weightVintage_90_in_24mob.json')
print("Model Loaded Successfully")

Model Loaded Successfully


In [3]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [4]:
combined_data = model_data.copy()

In [5]:
combined_data['90_in_24mob'].value_counts(dropna=False)

90_in_24mob
NaN    424397
0.0     44727
1.0      9204
Name: count, dtype: int64

In [6]:
combined_df= combined_data.drop_duplicates(subset='unique_reference_number')

In [7]:
combined_df= combined_df.drop_duplicates(subset='latest_feature_id')

In [8]:
df = combined_df.dropna(subset=['90_in_24mob'])

In [9]:
df.shape

(53931, 2831)

In [10]:
df['user_segment'].value_counts()

user_segment
OFFUS    38474
ONUS     15457
Name: count, dtype: int64

In [12]:
#take out onus records only
df_onus = df[df['user_segment']=='ONUS']

In [13]:
df_onus.shape

(15457, 2831)

In [14]:
#take out offus records only
df_offus= df[df['user_segment']=='OFFUS']

In [15]:
df_offus.shape

(38474, 2831)

In [17]:
#change the data records (df_offus - offus , df_onus - onus)
data = df_offus.copy()

In [18]:
data.shape

(38474, 2831)

In [19]:
import numpy as np
data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [20]:
#Split the time interval for model building and Validation of oot
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2022-06')]
oot_df1= data[(data['month_year']>='2022-07') & (data['month_year']<='2022-09')]

In [21]:
model_df1.shape, oot_df1.shape

((28167, 2831), (8572, 2831))

In [22]:
features_list =[
    
#Bureau Intutive features
'Num_Enq_all_loans_last_9M_3M',
'Utilization_all',
'total_emi',
'Utilization_CREDIT CARD_opened_last_36M',
'Utilization_PERSONAL LOAN_opened_last_9M',
'num_CREDIT CARD_enq_last_6M',
'num_CONSUMER LOAN_enq_last_36M',
'Num_of_PERSONAL LOAN_opened_last6M_12M',
'highest_sanction_amount_credit_cards',
'num_CONSUMER LOAN_enq_last_6M',
'num_PL_LE_1_Lac_opened_in_last_12M',
'Total_Sanction_amt_PERSONAL LOAN_Opened_last_24M',
'Age_of_oldest_open_account',
    
#Banking Intutive features

'num_credit_gt_salary',    
'upi_trans_count',
'ratio_balance_after_7_days_max_credit_M3',    
'ratio_max_credit_to_total_credit_M2',
'spend_vs_salary_ratio',
'bounced_transactions_count',
'ratio_total_debit_to_max_credit_M2_M3',
'min_balance_last_3_months',
'min_balance_M2',
'days_balance_lt_5000_M3',
'days_balance_lt_10000_M3',
'Max_Credit_Avg_M1_M2_M3',
'Max_Credit_Diff_M1_M2_M3',
'amount_debits_last_2_months',
'sum_insurance_debit',
'avg_num_debits_last_2_months',
'upi_trans_debit_sum',
'cash_withdrawal_sum',
    
#new banking features
'diff_btw_stated_salary_vs_actual_salary',
'balance_6th_day_M2',
'ratio_6th_day_balance_to_max_credit_M3',

#Loan application Features

'deviation_from_median_income/standard_deviation_income#last_365_Days',
# 'yearOfBirth_mean',
'loan_amount_to_income#last_365_Days',



#PG synthetic Features
'distinct_merchant',
'MEAN(raw.amount WHERE device_os_category = Windows)#ever',
'days_on_pg',
'NUM_UNIQUE(pg_success_raw_data.merchant_id WHERE revised_merchant_category = marketplace)',
'amt_first_android_txn', 
    
#ID Features
'dm206s',
'g406s',
'balmag01',

# # Reconcilated Features
'percentage_ever_delinquent', 
'months_since_most_recent_delinquency_installment', 
# 'months_since_most_recent_delinquency_all',
'percentage_utilization_gt_75',
'Num_of_installment_loan_opened_last_6M',
'No of enq in past 90days',
'Num_Enq_non_BANK_last_12M',
'month_year',
'90_in_24mob'
              
]




In [23]:
model_df = model_df1[features_list]

In [24]:
oot_df = oot_df1[features_list]

In [25]:
model_df.shape, oot_df.shape

((28167, 52), (8572, 52))

In [26]:
# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', '90_in_24mob']) # Adjust 'target' to your actual target column name
y_model = model_df['90_in_24mob']

In [27]:
X_oot = oot_df.drop(columns=['month_year', '90_in_24mob'])
y_oot = oot_df['90_in_24mob']

In [28]:
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3026315/3109673050.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)


In [29]:
for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3026315/1979277635.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_oot[i].fillna(-9999,inplace=True)


In [30]:
X_oot.shape,y_oot.shape

((8572, 50), (8572,))

In [31]:
# Split Model dataset into Train and Test
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [32]:
X_train.shape, X_test.shape

((16900, 50), (11267, 50))

In [33]:
# Create DMatrix for Train predictions
data_xtra_pred = xgb.DMatrix(X_train)
X_train['90_in_24mob'] = y_train
X_train['y_pred'] = loaded_model.predict(data_xtra_pred)

In [34]:
data_xt_pred = xgb.DMatrix(X_test)
X_test['90_in_24mob']= y_test
X_test['y_pred'] = loaded_model.predict(data_xt_pred)

In [35]:
# Create DMatrix for OOT predictions
data_x_pred = xgb.DMatrix(X_oot)
X_oot['90_in_24mob'] = y_oot
X_oot['y_pred'] = loaded_model.predict(data_x_pred)

In [36]:
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train, 'y_pred', '90_in_24mob')
test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test, 'y_pred', '90_in_24mob')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot, 'y_pred', '90_in_24mob')

In [63]:
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")

Train AUC: 0.69, Gini: 0.38, KS: 0.28
Test AUC: 0.69, Gini: 0.38, KS: 0.28
OOT AUC: 0.70, Gini: 0.40, KS: 0.29


In [38]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.419580         0.642695        0.799025        0.579826    858   
9          0.256709         0.543149        0.579775        0.509616    857   
8          0.221704         0.482173        0.509498        0.458596    857   
7          0.186698         0.439133        0.458521        0.421106    857   
6          0.158693         0.405519        0.421051        0.390942    857   
5          0.129522         0.374285        0.390934        0.358685    857   
4          0.103851         0.341629        0.358660        0.323181    857   
3          0.103851         0.304834        0.323140        0.287376    857   
2          0.078180         0.268885        0.287361        0.250085    857   
1          0.037296         0.225994        0.250076        0.168420    858   

        label_good  label_bad  
Decile                         
10             498        360  
9              637        220  
8              667        190  
7              697        160  
6              721        136  
5              746        111  
4              768         89  
3              768         89  
2              790         67  
1              826         32

In [39]:
feature_importance = loaded_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                              Feature  Importance
38                                         days_on_pg       132.0
42                                              g406s       120.0
0                        Num_Enq_all_loans_last_9M_3M        73.0
12                         Age_of_oldest_open_account        72.0
49                          Num_Enq_non_BANK_last_12M        66.0
37  MEAN(raw.amount WHERE device_os_category = Win...        58.0
1                                     Utilization_all        56.0
18                         bounced_transactions_count        54.0
36                                  distinct_merchant        53.0
40                              amt_first_android_txn        51.0
43                                           balmag01        43.0
4            Utilization_PERSONAL LOAN_opened_last_9M        43.0
34  deviation_from_median_income/standard_deviatio...        40.0
8                highest_sanction_amount_credit_cards        38.0
48        

In [40]:
feature_importance_df

Feature  Importance
38                                         days_on_pg       132.0
42                                              g406s       120.0
0                        Num_Enq_all_loans_last_9M_3M        73.0
12                         Age_of_oldest_open_account        72.0
49                          Num_Enq_non_BANK_last_12M        66.0
37  MEAN(raw.amount WHERE device_os_category = Win...        58.0
1                                     Utilization_all        56.0
18                         bounced_transactions_count        54.0
36                                  distinct_merchant        53.0
40                              amt_first_android_txn        51.0
43                                           balmag01        43.0
4            Utilization_PERSONAL LOAN_opened_last_9M        43.0
34  deviation_from_median_income/standard_deviatio...        40.0
8                highest_sanction_amount_credit_cards        38.0
48                           No of enq in past 90days        38.0
44                         percentage_ever_delinquent        38.0
14                                    upi_trans_count        36.0
9                       num_CONSUMER LOAN_enq_last_6M        34.0
7              Num_of_PERSONAL LOAN_opened_last6M_12M        30.0
30                                cash_withdrawal_sum        29.0
39  NUM_UNIQUE(pg_success_raw_data.merchant_id WHE...        27.0
5                         num_CREDIT CARD_enq_last_6M        25.0
10                 num_PL_LE_1_Lac_opened_in_last_12M        21.0
23                           days_balance_lt_10000_M3        21.0
22                            days_balance_lt_5000_M3        21.0
6                      num_CONSUMER LOAN_enq_last_36M        20.0
47             Num_of_installment_loan_opened_last_6M        19.0
45   months_since_most_recent_delinquency_installment        19.0
35                loan_amount_to_income#last_365_Days        19.0
13                               num_credit_gt_salary        18.0
2                                           total_emi        17.0
3             Utilization_CREDIT CARD_opened_last_36M        17.0
11   Total_Sanction_amt_PERSONAL LOAN_Opened_last_24M        16.0
46                       percentage_utilization_gt_75        15.0
28                       avg_num_debits_last_2_months        14.0
33             ratio_6th_day_balance_to_max_credit_M3        13.0
15           ratio_balance_after_7_days_max_credit_M3        13.0
27                                sum_insurance_debit         9.0
21                                     min_balance_M2         9.0
41                                             dm206s         9.0
32                                 balance_6th_day_M2         8.0
17                              spend_vs_salary_ratio         8.0
20                          min_balance_last_3_months         7.0
24                            Max_Credit_Avg_M1_M2_M3         7.0
16                ratio_max_credit_to_total_credit_M2         6.0
19              ratio_total_debit_to_max_credit_M2_M3         5.0
29                                upi_trans_debit_sum         4.0
25                           Max_Credit_Diff_M1_M2_M3         3.0
31            diff_btw_stated_salary_vs_actual_salary         3.0
26                        amount_debits_last_2_months         2.0